In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import os

In [9]:
import os
import pandas as pd

metadata = pd.read_csv('male_train.csv')
image_dir = 'images/all_images/'
image_files = set(os.listdir(image_dir))

BCC_count = 0
SCC_count = 0
ACK_count = 0
NEV_count = 0
SEK_count = 0
MEL_count = 0
total = 0

# Iterate over the entire DataFrame
for diagnostic in metadata['diagnostic']:
    if diagnostic == 'BCC':
        BCC_count += 1
    elif diagnostic == 'SCC':
        SCC_count += 1
    elif diagnostic == 'ACK':
        ACK_count += 1
    elif diagnostic == 'NEV':
        NEV_count += 1
    elif diagnostic == 'SEK':
        SEK_count += 1
    elif diagnostic == 'MEL':
        MEL_count += 1
    else:
        total -= 1
    total += 1

# drop the random 620 rows that are diognostic as BCC
metadata = metadata[metadata['diagnostic'] != 'BCC']

print(BCC_count, SCC_count, MEL_count, ACK_count, NEV_count, SEK_count, total)
print(MEL_count + BCC_count + SCC_count, SEK_count + NEV_count + ACK_count)


100 24 4 96 21 13 258
128 130


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset
metadata = pd.read_csv('final_dataset.csv')
print(f"Original metadata length: {len(metadata)}")

# Check for missing values in the gender column
missing_gender = metadata[metadata['gender'] == ' ']
#missing_gender_count = missing_gender.sum()
#print(f"Missing gender values: {missing_gender_count}")

# Filter the data by gender
maledata = metadata[metadata['gender'] == 'MALE']
femaledata = metadata[metadata['gender'] == 'FEMALE']

# Print lengths of filtered data
print(f"Male data length: {len(maledata)}")
print(f"Female data length: {len(femaledata)}")

# Split the male data into training and testing sets
male_train, male_test = train_test_split(maledata, test_size=0.25, random_state=42)

# Split the female data into training and testing sets
female_train, female_test = train_test_split(femaledata, test_size=0.25, random_state=42)

# Print lengths of splits
print(f"Male train length: {len(male_train)}")
print(f"Male test length: {len(male_test)}")
print(f"Female train length: {len(female_train)}")
print(f"Female test length: {len(female_test)}")

# Verify total length
total_split_length = len(male_train) + len(male_test) + len(female_train) + len(female_test)
print(f"Total length after splitting: {total_split_length}")

# Save the datasets to separate CSV files
missing_gender.to_csv('missing.csv', index=False)
male_train.to_csv('male_train.csv', index=False)
male_test.to_csv('male_test.csv', index=False)
female_train.to_csv('female_train.csv', index=False)
female_test.to_csv('female_test.csv', index=False)


Original metadata length: 690
Male data length: 345
Female data length: 345
Male train length: 258
Male test length: 87
Female train length: 258
Female test length: 87
Total length after splitting: 690


In [45]:
import pandas as pd
import numpy as np

# Read the dataset
metadata = pd.read_csv('missing.csv')

# Separate rows by diagnosis
bcc_rows = metadata[metadata['diagnostic'] == 'BCC']
ack_rows = metadata[metadata['diagnostic'] == 'ACK']
other_rows = metadata[(metadata['diagnostic'] != 'BCC') & (metadata['diagnostic'] != 'ACK')]

# Calculate the number of rows to drop for BCC and ACK
bcc_to_drop = int(len(bcc_rows) * 3 / 4)
ack_to_drop = int(len(ack_rows) * 5 / 7)

# Randomly drop the rows
bcc_rows_dropped = bcc_rows.sample(n=bcc_to_drop, random_state=42)
ack_rows_dropped = ack_rows.sample(n=ack_to_drop, random_state=42)

# Get the remaining rows
bcc_rows_remaining = bcc_rows.drop(bcc_rows_dropped.index)
ack_rows_remaining = ack_rows.drop(ack_rows_dropped.index)

# Concatenate the remaining rows to form the balanced dataset
balanced_metadata = pd.concat([bcc_rows_remaining, ack_rows_remaining, other_rows])

# Shuffle the balanced dataset
balanced_metadata = balanced_metadata.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the new counts for verification
print(balanced_metadata['diagnostic'].value_counts())

# Save the balanced dataset
balanced_metadata.to_csv('balanced_metadata.csv', index=False)

diagnostic
SEK    188
NEV    169
ACK    128
Name: count, dtype: int64


In [48]:
import pandas as pd

# Read the dataset
metadata = pd.read_csv('metadata.csv')
print(f"Original metadata length: {len(metadata)}")

# Remove rows with missing gender
metadata = metadata.dropna(subset=['gender'])
print(f"Metadata length after removing rows with missing gender: {len(metadata)}")

# Filter the data by gender
maledata = metadata[metadata['gender'] == 'MALE']
femaledata = metadata[metadata['gender'] == 'FEMALE']

# Define the function to drop a specific number of rows according to the given ratio
def drop_rows_by_ratio(data, drop_count, drop_ratio, keep_ratio):
    total_rows = len(data)
    indices_to_drop = []
    dropped_count = 0

    i = 0
    while dropped_count < drop_count:
        if i + drop_ratio + keep_ratio <= total_rows and dropped_count + drop_ratio <= drop_count:
            indices_to_drop.extend(range(i, i + drop_ratio))
            dropped_count += drop_ratio
            i += drop_ratio + keep_ratio
        else:
            # If close to the end, drop remaining needed rows
            remaining_to_drop = drop_count - dropped_count
            indices_to_drop.extend(range(i, i + remaining_to_drop))
            break

    return data.drop(data.index[indices_to_drop])

# Drop 396 male rows using the specified ratio
male_data_dropped = drop_rows_by_ratio(maledata, 396, 4, 3)
print(f"Length of male data after dropping rows: {len(male_data_dropped)}")

# Drop 408 female rows using the specified ratio
female_data_dropped = drop_rows_by_ratio(femaledata, 408, 4, 3)
print(f"Length of female data after dropping rows: {len(female_data_dropped)}")

# Verify the number of rows dropped
print(f"Number of male rows dropped: {len(maledata) - len(male_data_dropped)}")
print(f"Number of female rows dropped: {len(femaledata) - len(female_data_dropped)}")

# Save the datasets to separate CSV files
male_data_dropped.to_csv('male_data_dropped.csv', index=False)
female_data_dropped.to_csv('female_data_dropped.csv', index=False)



Original metadata length: 2298
Metadata length after removing rows with missing gender: 2298
Length of male data after dropping rows: 345
Length of female data after dropping rows: 345
Number of male rows dropped: 396
Number of female rows dropped: 408


In [2]:
import pandas as pd

# Read the dataset
metadata = pd.read_csv('metadata.csv')
print(f"Original metadata length: {len(metadata)}")

# Remove rows with missing gender (represented as empty string ' ')
metadata = metadata[metadata['gender'] != ' ']
print(f"Metadata length after removing rows with missing gender: {len(metadata)}")

# Filter the data by gender
maledata = metadata[metadata['gender'] == 'MALE']
femaledata = metadata[metadata['gender'] == 'FEMALE']

# Define the function to drop specific numbers of rows based on diagnostic values
def drop_specific_rows(data, drop_counts):
    indices_to_drop = []
    for diagnostic, count in drop_counts.items():
        diagnostic_indices = data[data['diagnostic'] == diagnostic].index[:count]
        indices_to_drop.extend(diagnostic_indices)
    return data.drop(indices_to_drop)

# Drop 396 male rows: 19 MEL, 308 BCC, 69 SCC
male_drop_counts = {'MEL': 16, 'BCC': 306, 'SCC': 71, 'ACK': 3}
male_data_dropped = drop_specific_rows(maledata, male_drop_counts)
print(f"Length of male data after dropping rows: {len(male_data_dropped)}")

# Drop 408 female rows: 20 MEL, 318 BCC, 70 SCC
female_drop_counts = {'MEL': 19, 'BCC': 275, 'SCC': 57, 'ACK': 37, 'NEV': 13, 'SEK': 7}
female_data_dropped = drop_specific_rows(femaledata, female_drop_counts)
print(f"Length of female data after dropping rows: {len(female_data_dropped)}")

# Verify the number of rows dropped
print(f"Number of male rows dropped: {len(maledata) - len(male_data_dropped)}")
print(f"Number of female rows dropped: {len(femaledata) - len(female_data_dropped)}")

# Concatenate the resulting datasets
final_dataset = pd.concat([male_data_dropped, female_data_dropped], ignore_index=True)
print(f"Length of final dataset after concatenation: {len(final_dataset)}")

# Save the final concatenated dataset to a CSV file
final_dataset.to_csv('final_dataset.csv', index=False)


Original metadata length: 2298
Metadata length after removing rows with missing gender: 2298
Length of male data after dropping rows: 345
Length of female data after dropping rows: 345
Number of male rows dropped: 396
Number of female rows dropped: 408
Length of final dataset after concatenation: 690
